### Spin Test 
This script projects data to surface space and performs a spin test

#### Import Packages

In [75]:
import numpy as np
import pandas as pd
import netneurotools.stats as stats
from neuromaps.datasets import fetch_atlas
from nilearn.datasets import fetch_atlas_schaefer_2018
from netneurotools.datasets import fetch_schaefer2018
import neuromaps
import nibabel as nib
from neuromaps.parcellate import Parcellater
from neuromaps import transforms
from neuromaps import nulls

#### Set Paths and Variables

In [53]:
dataset = 'longglucest_outputmeasures2'
atlas = 'Schaefer2018_1000Parcels_17Networks'
nmaps = ["NMDA", "mGluR5", "GABA"]
maps = ["cest", "NMDA", "mGluR5", "GABA"]

# Set paths
inpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/nmaps/analyses/" + atlas
outpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/nmaps/analyses/" + atlas
means= pd.read_csv(inpath + '/means_subjectnmaps_' + dataset + '_' + atlas + '.csv', sep=',')
means["Parcel"] = means["Parcel"].str.replace(' NZMean', '', regex=False)
means.set_index('Parcel', inplace = True)

# Fetch Schaefer 2018 atlas coordinates (for example)
schaefer = fetch_atlas_schaefer_2018(n_rois=1000, yeo_networks=17)
labels = schaefer['labels']  # These are the MNI coordinates of the parcels
labels = [label.decode('utf-8') for label in labels]
# Load in Schaefer nifti.
schaefernifti = nib.load(schaefer['maps'])
schaeferarray = schaefernifti.get_fdata()
print(schaeferarray.shape)

print(means)

(182, 218, 182)
                                 hstatus  GluCEST_avg  NMDA_avg  mGluR5_avg  \
Parcel                                                                        
17Networks_RH_ContA_Cingm_1           HC     7.348242 -0.515943   -0.290452   
17Networks_RH_ContA_Cingm_1          PSY     7.536327 -0.478070   -0.286724   
17Networks_RH_ContA_Cingm_2           HC     7.400822 -0.662924   -0.352973   
17Networks_RH_ContA_Cingm_2          PSY     7.629667 -0.608125   -0.341912   
17Networks_RH_ContB_PFCmp_1           HC     7.561262  0.665146    0.898630   
...                                  ...          ...       ...         ...   
17Networks_RH_VisPeri_ExStrSup_7     PSY     8.500733  0.819882    1.357389   
17Networks_RH_VisPeri_StriCal_3       HC     9.569096 -0.013643    0.141399   
17Networks_RH_VisPeri_StriCal_3      PSY     9.336277  0.003004    0.171412   
17Networks_RH_VisPeri_StriCal_5       HC     9.077889  0.177231    0.461898   
17Networks_RH_VisPeri_StriCal_5     

### Convert array data to nifti then to surface data

In [69]:
# Assuming GluCEST_data and NMDA_data are 1D arrays with data for each parcel
means = means[means['hstatus']== 'HC']

# Initialize an array with NaNs the same length as the labels
glucest_array = np.full(len(labels), np.nan)
nmda_array = np.full(len(labels), np.nan)
gaba_array = np.full(len(labels), np.nan)
mglur5_array = np.full(len(labels), np.nan)

# Loop through the labels and fill in the corresponding GluCEST_avg values where they exist in the dataframe
for i, label in enumerate(labels):
    if label in means.index:
        #print(label)
        glucest_array[i] = means.loc[label, 'GluCEST_avg']
        nmda_array[i] = means.loc[label, 'NMDA_avg']
        gaba_array[i] = means.loc[label, 'GABA_avg']
        mglur5_array[i] = means.loc[label, 'mGluR5_avg']

# Reshape Arrays
glucest_array = glucest_array.reshape(1, len(labels))
nmda_array = nmda_array.reshape(1, len(labels))
gaba_array = gaba_array.reshape(1, len(labels))
mglur5_array = mglur5_array.reshape(1, len(labels))

# Parcellate and convert back to nifti
parcellater = Parcellater(schaefer['maps'], 'MNI152')
glucest_nifti = parcellater.inverse_transform(glucest_array)
nmda_nifti = parcellater.inverse_transform(nmda_array)
gaba_nifti = parcellater.inverse_transform(gaba_array)
mglur5_nifti = parcellater.inverse_transform(mglur5_array)

# Save avg niftis
nib.save(glucest_nifti,outpath +'avg_glucest.nii.gz')
nib.save(nmda_nifti,outpath +'avg_nmda.nii.gz')
nib.save(gaba_nifti,outpath +'avg_gaba.nii.gz')
nib.save(mglur5_nifti,outpath +'avg_mglur5.nii.gz')


#### Project to surface space

In [78]:
fs_glucest = transforms.mni152_to_fslr(nmda_nifti, '32k')
fs_nmda = transforms.mni152_to_fslr(nmda_nifti, '32k')
fs_gaba = transforms.mni152_to_fslr(gaba_nifti, '32k')
fs_mglur5 = transforms.mni152_to_fslr(mglur5_nifti, '32k')

fsavg_lh, fsavg_rh = fsavg
print(fs_glucest)
print(fsavg_lh.agg_data().shape)

rotated = nulls.alexander_bloch(fs_glucest, atlas='fslr', density='32k',
                                n_perm=10, seed=1234, parcellation=fs_glucest)


(<nibabel.gifti.gifti.GiftiImage object at 0x35b778f50>, <nibabel.gifti.gifti.GiftiImage object at 0x35b779190>)
(163842,)


/Users/pecsok/miniforge3/envs/nmaps2/lib/python3.11/site-packages/neuromaps/nulls/spins.py:132: RuntimeWarning: Mean of empty slice.
  roi = np.atleast_2d(vertices[mask].mean(axis=0))
/Users/pecsok/.local/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
from neuromaps import parcellate
destrieux = parcellate.Parcellater(parcellation, 'fsaverage').fit()
neurosynth_parc = destrieux.transform(neurosynth, 'fsaverage')
abagen_parc = destrieux.transform(abagen, 'fsaverage')
print(neurosynth_parc.shape, abagen_parc.shape)


rotated = nulls.alexander_bloch(neurosynth_parc, atlas='fsaverage', density='10k',
                                n_perm=100, seed=1234, parcellation=parcellation)
print(rotated.shape)

In [ ]:
# Warp each subject's parcellated data? 
# We have:
#    Parcellated atlas file in subject space
#    Subject-MNI transform
#    Average parcel value   
#    Standardize warp from MNI to fsLR or fsavg space

# Goal: 
#    Use 

# 1. Repopulate parcels in subject slice with average parcel value

### Inputs: subject slice nifi, CSV with parcels/values,

# 2. Project that to surface 